In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import SelectKBest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score,recall_score,precision_score, roc_curve, auc, confusion_matrix, classification_report,accuracy_score

In [ ]:
data = pd.read_csv('../input/ckdisease/kidney_disease.csv')

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.isnull().any()

In [ ]:
data.isna().sum()

In [ ]:
NewCols={"bp":"blood_pressure","sg":"specific_gravity", "al":"albumin","su":"sugar","rbc":"red_blood_cells","pc":"pus_cell",
         "pcc":"pus_cell_clumps","ba":"bacteria","bgr":"blood_glucose_random","bu":"blood_urea","sc":"serum_creatinine",
         "sod":"sodium","pot":"potassium","hemo":"haemoglobin","pcv":"packed_cell_volume","wc":"white_blood_cell_count",
          "rc":"red_blood_cell_count","htn":"hypertension","dm":"diabetes_mellitus","cad":"coronary_artery_disease",
          "appet":"appetite","pe":"pedal_edema","ane":"anemia"}

In [ ]:
data.rename(columns=NewCols, inplace=True)

In [ ]:
data.count()

In [ ]:
data.describe()

In [ ]:
#Null values -  Bar Graphs
plt.subplots(figsize=(10, 7))
((data.isnull().sum())).sort_values(ascending=False).plot(kind='bar')
plt.show()

In [ ]:
#Converting to numerics
data[['red_blood_cells','pus_cell']] = data[['red_blood_cells','pus_cell']].replace(to_replace={'abnormal':1,'normal':0})
data[['pus_cell_clumps','bacteria']] = data[['pus_cell_clumps','bacteria']].replace(to_replace={'present':1,'notpresent':0})
data[['hypertension','diabetes_mellitus','coronary_artery_disease','pedal_edema','anemia']] = data[['hypertension','diabetes_mellitus','coronary_artery_disease','pedal_edema','anemia']].replace(to_replace={'yes':1,'no':0})
data[['appetite']] = data[['appetite']].replace(to_replace={'good':1,'poor':0,'no':np.nan})
data['coronary_artery_disease'] = data['coronary_artery_disease'].replace(to_replace='\tno',value=0)
data['diabetes_mellitus'] = data['diabetes_mellitus'].replace(to_replace={'\tno':0,'\tyes':1,' yes':1, '':np.nan})
data['classification'] = data['classification'].replace(to_replace={'ckd':1.0,'ckd\t':1.0,'notckd':0.0,'no':0.0})
data['pedal_edema'] = data['pedal_edema'].replace(to_replace='good',value=0) 
data['appetite'] = data['appetite'].replace(to_replace='no',value=0)
data['coronary_artery_disease']=data['coronary_artery_disease'].replace('yes',1)

In [ ]:
data.drop(["id"],inplace=True,axis=1)

In [ ]:
#Treating Special chars and NaNs and missing values
data = data.replace("\t?", np.nan)
data = data.replace(" ?", np.nan)
data = data.fillna(method='ffill')
data = data.fillna(method='backfill')

In [ ]:
data['age']=data['age'].fillna(np.mean(data['age']))
data['blood_pressure']=data['blood_pressure'].fillna(np.mean(data['blood_pressure']))
data['albumin']=data['albumin'].fillna(np.mean(data['albumin']))
data['specific_gravity']=data['specific_gravity'].fillna(np.mean(data['specific_gravity']))
data['sugar']=data['sugar'].fillna(np.mean(data['sugar']))
data['blood_glucose_random']=data['blood_glucose_random'].fillna(np.mean(data['blood_glucose_random']))
data['blood_urea']=data['blood_urea'].fillna(np.mean(data['blood_urea']))
data['serum_creatinine']=data['serum_creatinine'].fillna(np.mean(data['serum_creatinine']))
data['haemoglobinhaemoglobin']=data['haemoglobin'].fillna(np.mean(data['haemoglobin']))
data['potassium']=data['potassium'].fillna(np.mean(data['potassium']))
data['sodium']=data['sodium'].fillna(np.mean(data['sodium']))

In [ ]:
data.isnull().any().sum()

In [ ]:
data.isna().sum()

In [ ]:
#checking Outliers
fig, ax = plt.subplots()
ax.scatter(x = data['specific_gravity'], y = data['classification'])
plt.ylabel('specific_gravity', fontsize=13)
plt.xlabel('classfication', fontsize=13)
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x = data['sugar'], y = data['classification'])
plt.ylabel('sugar', fontsize=13)
plt.xlabel('classfication', fontsize=13)
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x = data['blood_pressure'], y = data['classification'])
plt.ylabel('blood_pressure', fontsize=13)
plt.xlabel('classification', fontsize=13)
plt.show()

In [ ]:
categoricalFeatures = data.select_dtypes(include='object')
numericalFeatures = data.select_dtypes(include=np.number)

In [ ]:
numcor=numericalFeatures.corr()
catcor=categoricalFeatures.corr()

In [ ]:
sns.pairplot(numcor)
plt.show()

In [ ]:
sns.heatmap(datacorrnumerical,annot=True)
plt.show()

In [ ]:
sns.boxplot(x='hypertension', y='albumin', data=data, palette='viridis')
plt.show()

In [ ]:
sns.boxplot(x='hypertension', y='specific_gravity', data=data, palette='viridis')
plt.show()

In [ ]:
corr_df = data.corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr_df, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(250, 15, as_cmap=False)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr_df, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.title('Correlations between different predictors')
plt.show()

In [ ]:
def auc_scorer(clf, X, y, model): # Helper function to plot the ROC curve
    if model=='RF':
        fpr, tpr, _ = roc_curve(y, clf.predict_proba(X)[:,1])
    elif model=='SVM':
        fpr, tpr, _ = roc_curve(y, clf.decision_function(X))
    roc_auc = auc(fpr, tpr)

    plt.figure()    # Plot the ROC curve
    plt.plot(fpr, tpr, label='ROC curve from '+model+' model (area = %0.3f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc="lower right")
    plt.show()

    return fpr,tpr,roc_auc

In [ ]:
#Split Model
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:,:-1], data['classification'], 
                                                    test_size = 0.3, random_state=100,
                                                   stratify= data['classification'] )

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
y_train.value_counts()

In [ ]:
#GridSearch CV
tuned_parameters = [{'n_estimators':[7,8,9,10,11,12,13,14,15,16],'max_depth':[2,3,4,5,6,None],
                     'class_weight':[None,{0: 0.33,1:0.67},'balanced'],'random_state':[100]}]
clf = GridSearchCV(RandomForestClassifier(), tuned_parameters, cv=10,scoring='f1')
clf.fit(X_train, y_train)

print("Detailed classification report:")
y_true, lr_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, lr_pred))

confusion = confusion_matrix(y_test, lr_pred)
print('Confusion Matrix:')
print(confusion)

# Determine the false positive and true positive rates
fpr,tpr,roc_auc = auc_scorer(clf, X_test, y_test, 'RF')

print('Best parameters:')
print(clf.best_params_)
clf_best = clf.best_estimator_

In [ ]:
plt.figure(figsize=(12,3))
features = X_test.columns.values.tolist()
importance = clf_best.feature_importances_.tolist()
feature_series = pd.Series(data=importance,index=features)
feature_series.plot.bar()
plt.title('Feature Importance')
plt.show()

In [ ]:
#KNN 
from sklearn.neighbors import KNeighborsClassifier
modelknn = KNeighborsClassifier(n_neighbors=7)
modelknn.fit(X_train, y_train)
y_predknn = modelknn.predict(X_test)


In [ ]:
print(classification_report(y_test, y_predknn))

In [ ]:
CMknn=confusion_matrix(y_test, y_predknn)
CMknn

In [ ]:
accuracy = accuracy_score(y_test,y_predknn)
print('Accuracy: %f' % accuracy)
accuracy = balanced_accuracy_score(y_test,y_predknn)
print('Balanced_Accuracy: %f' % accuracy)
precision = precision_score(y_test,y_predknn)
print('Precision: %f' % precision)
recall = recall_score(y_test,y_predknn)
print('Recall: %f' % recall)
f1 = f1_score(y_test,y_predknn)
print('F1 score: %f' % f1)

In [ ]:
#Decision Tree
modeldt = DecisionTreeClassifier()
modeldt.fit(X_train, y_train)

In [ ]:
y_preddt = modeldt.predict(X_test)
y_preddt

In [ ]:
CMDT=confusion_matrix(y_test,y_preddt)
CMDT

In [ ]:
print(classification_report(y_test, y_preddt))

In [ ]:
accuracy = accuracy_score(y_test,y_preddt)
print('Accuracy: %f' % accuracy)
accuracy = balanced_accuracy_score(y_test,y_preddt)
print('Balanced_Accuracy: %f' % accuracy)
precision = precision_score(y_test,y_preddt)
print('Precision: %f' % precision)
recall = recall_score(y_test,y_preddt)
print('Recall: %f' % recall)
f1 = f1_score(y_test,y_preddt)
print('F1 score: %f' % f1)

In [ ]:
#Standard Scalar
scaler = StandardScaler()
scaler.fit(X_train)

In [ ]:
x_train = scaler.transform(X_train)
x_test = scaler.transform(X_test)

In [ ]:
modelRF = RandomForestClassifier()
modelRF.fit(X_train, y_train)

In [ ]:
y_predrf = modelRF.predict(X_test)
y_predrf

In [ ]:
print(classification_report(y_test, y_predrf))

In [ ]:
CMRF=confusion_matrix(y_test, y_predrf)
CMRF

In [ ]:
accuracy = accuracy_score(y_test,y_predrf)
print('Accuracy: %f' % accuracy)
accuracy = balanced_accuracy_score(y_test,y_predrf)
print('Balanced_Accuracy: %f' % accuracy)
precision = precision_score(y_test,y_predrf)
print('Precision: %f' % precision)
recall = recall_score(y_test,y_predrf)
print('Recall: %f' % recall)
f1 = f1_score(y_test,y_predrf)
print('F1 score: %f' % f1)